In [1]:
import keras

from keras.layers import Dense
from keras.models import Model

import h5py

import numpy as np
import os

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization, Activation
from keras import optimizers

Using TensorFlow backend.


#### [Allowing GPU memory growth](https://www.tensorflow.org/programmers_guide/using_gpu#allowing_gpu_memory_growth)
```
...\stream_executor\cuda\cuda_dnn.cc:371] could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
...\stream_executor\cuda\cuda_dnn.cc:375] error retrieving driver version: Unimplemented: kernel reported driver version not implemented on Windows
...\stream_executor\cuda\cuda_dnn.cc:338] could not destroy cudnn handle: CUDNN_STATUS_BAD_PARAM
```

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [3]:
with h5py.File('datasets/inception_resnet_v2.h5', 'r') as f:
    train_x = np.array(f['train_features'])
    train_y = np.array(f['train_labels'])
    val_x = np.array(f['val_features'])
    val_y = np.array(f['val_labels'])
    test_x = np.array(f['test_features'])

KeyError: "Unable to open object (object 'test_features' doesn't exist)"

In [4]:
with h5py.File('datasets/inception_resnet_v2_test.h5', 'r') as f:
    test_x = np.array(f['test_features'])

In [5]:
inputs = Input(train_x.shape[1:])
x = inputs
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(.5)(x)
y = Dense(120, activation='softmax')(x)

model = Model(inputs=inputs, outputs=y)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8, 8, 1536)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               393472    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               30840     
Total params: 424,312
Trainable params: 424,312
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.save_weights('empty_weights.h5')

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(
    x=train_x,
    y=train_y,
    batch_size=32,
    epochs=1,
    validation_data=(val_x, val_y)
)

Train on 8221 samples, validate on 2001 samples
Epoch 1/1
8221/8221 [==============================] - 6s 675us/step - loss: 0.4873 - acc: 0.8640 - val_loss: 0.3043 - val_acc: 0.9110


In [16]:
model.load_weights('empty_weights.h5')

In [12]:
model.save_weights('good_weights.h5')

In [13]:
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image

def train_loader(path, shuffle=True, batch_size=32, target_size=(224,224), preprocessing_function=preprocess_input):
    datagen = image.ImageDataGenerator(
        preprocessing_function=preprocessing_function
    )

    return datagen.flow_from_directory(
        path,
        target_size=target_size,
        batch_size=batch_size, 
        shuffle=shuffle,
    )

loader = train_loader('datasets/val')
classes = list(loader.class_indices.keys())

Found 2001 images belonging to 120 classes.


In [14]:
y_preds = model.predict(test_x, batch_size=32)

In [15]:
test_loader = train_loader('datasets/test')

Found 10357 images belonging to 1 classes.


In [16]:
images_id = [f[8:-4] for f in test_loader.filenames]

In [17]:
import pandas as pd

df = pd.DataFrame(y_preds)

df.columns = classes

df.insert(0, 'id', images_id)

In [18]:
df.to_csv('sub.gz', compression='gzip', index=False)